# 파마 인디언 당뇨병 예측

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import files
up = files.upload()

Saving pima-indians-diabetes.csv to pima-indians-diabetes.csv


- 데이터 전처리

In [4]:
df = pd.read_csv("pima-indians-diabetes.csv",skiprows=9,header=None)
df

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [5]:
df.columns = ['P',"G","BP","S","I","BMI","D","Age","Class"]
df.head()

,P,G,BP,S,I,BMI,D,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
# 결측치 확인
df.isna().sum()

P        0
G        0
BP       0
S        0
I        0
BMI      0
D        0
Age      0
Class    0
dtype: int64

In [8]:
# X,y를 ndarray로 추출
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [9]:
X.shape,y.shape

((768, 8), (768,))

- Train/Test dataset으로 분리

In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
    X,y,stratify = y, test_size=0.2, random_state=2022
)

In [27]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((614, 8), (154, 8), (614,), (154,))

In [28]:
np.unique(y_train,return_counts=True)

(array([0, 1]), array([400, 214]))

In [29]:
np.unique(y_test,return_counts=True)

(array([0, 1]), array([100,  54]))

- Decision Tree Classifier

In [30]:
from sklearn.tree import DecisionTreeClassifier

In [31]:
dtc = DecisionTreeClassifier(random_state=2022)

In [32]:
dtc.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 2022,
 'splitter': 'best'}

In [35]:
params = {
    'max_depth':[2,3,4,5,6],'min_samples_split':[2,3,4]
}

GridSearchCV

In [36]:
from sklearn.model_selection import GridSearchCV
grid_dt = GridSearchCV(dtc,params,scoring='accuracy',cv=5)
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2022),
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [37]:
grid_dt.best_params_

{'max_depth': 5, 'min_samples_split': 3}

In [38]:
best_dt = grid_dt.best_estimator_
best_dt.score(X_test,y_test)

0.6558441558441559

In [39]:
X_test[0],y_test[0]

(array([  0.   , 135.   ,  94.   ,  46.   , 145.   ,  40.6  ,   0.284,
         26.   ]), 0)

In [40]:
X_test.shape,X_test[0].shape #2차원과 1차원으로 차원이 다름

((154, 8), (8,))

In [41]:
best_dt.predict(X_test[0].reshape(1,-1)) #차원 맞추기

array([0])

In [42]:
pred = best_dt.predict(X_test[0].reshape(1,-1))
print("양성" if pred[0] == 1 else "음성")

음성
